<a href="https://colab.research.google.com/github/pam-lab/JupyterFiles/blob/main/NLP_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

In [ ]:
!pip install -q transformers
!pip install -qU hazm
!pip install datasets
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd

import hazm

import transformers 
from transformers import AutoTokenizer, AutoConfig
from transformers import TFAutoModelForTokenClassification
import itertools
from datasets import Dataset
from datasets import load_metric
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

import os
from IPython.display import display, HTML, clear_output
from ipywidgets import widgets, Layout

print()
print('tensorflow', tf.__version__)
print('transformers', transformers.__version__)
print('numpy', np.__version__)
print('pandas', pd.__version__)
print()

if tf.test.gpu_device_name() != '/device:GPU:0':
    print()
    print('WARNING: GPU device not found.')
else:
    print()
    print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))


tensorflow 2.8.2
transformers 4.21.0
numpy 1.21.6
pandas 1.3.5


SUCCESS: Found GPU: /device:GPU:0


### Fine-tune model with our data

In [ ]:
!gdown 1AzXg9YVaXCAzalr57IpP1sNbWBBbZrqI

Downloading...
From: https://drive.google.com/uc?id=1AzXg9YVaXCAzalr57IpP1sNbWBBbZrqI
To: /content/ner.csv
100% 383k/383k [00:00<00:00, 94.1MB/s]


In [ ]:
data = pd.read_csv('ner.csv', )
data

,token,labels
0,"['حال', 'و', 'هوای', 'حرم', 'امام', 'رضا', '(ع...","['o', 'o', 'o', 'o', 'o', 'B-PER', 'o', 'o', '..."
1,"['به', 'گزارش', 'خبرگزاری', 'تسنیم', 'از', 'مش...","['o', 'o', 'B-ORG', 'I-ORG', 'o', 'B-LOC', 'o'..."
2,"['به', 'گزارش', 'خبرگزاری', 'تسنیم', 'از', 'مش...","['o', 'o', 'B-ORG', 'I-ORG', 'o', 'B-LOC', 'o'..."
3,"['انریکه:', 'سرزنش', 'دخه\u200cآ', 'عادت', 'بد...","['B-PER', 'o', 'B-PER', 'o', 'o', 'o', 'o', 'o..."
4,"['به', 'گزارش،', 'لوئیس', 'انریکه', 'که', 'شاگ...","['o', 'o', 'B-PER', 'I-PER', 'o', 'o', 'B-DAT'..."
...,...,...
220,"['به', 'گزارش', 'شبکه', 'اطلاع', 'رسانی', 'راه...","['o', 'o', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG',..."
221,"['به', 'گزارش', 'شبکه', 'اطلاع', 'رسانی', 'راه...","['o', 'o', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG',..."
222,"['لزوم', 'تشکیل', 'قرارگاه', 'عملیاتی', 'مانع'...","['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']"
223,"['وی', 'با', 'اشاره', 'به', 'نام', 'گذاری', 'ا...","['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', ..."


In [ ]:
def clean_alt_list(list_):
    list_ = list_.replace('[','').replace(']','').split(',')
    return [item.replace("'","").strip() for item in list_]

In [ ]:
data['token'] = data['token'].apply(clean_alt_list)
data['labels'] = data['labels'].apply(clean_alt_list)

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(data)

### new Way

In [ ]:
tag_scheme = ["B-DAT", "B-TIM", "B-PER", "B-ORG", "B-LOC", "B-EVE", "B-NAT", "B-mainLOC", "I-DAT", "I-TIM", "I-PER", "I-ORG", "I-LOC", "I-EVE", "I-NAT", "I-mainLOC", "o"]

In [ ]:
label_list = tag_scheme
label_encoding_dict = {'I-DAT':1,'I-EVE':2,'o': 0,'I-LOC':3,'I-ORG':4,'I-PER':5,'I-TIM':6,'I-mainLOC':7 ,'I-NAT':8,'B-DAT':9,'B-EVE':10,'B-LOC':11,'B-ORG':12,'B-PER':13,'B-TIM':14,'B-mainLOC':15,'B-NAT':16}

In [ ]:
batch_size = 16
task = "ner" 
model_name = 'HooshvareLab/bert-base-parsbert-ner-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
test_size=.15

processed_dataset = dataset.shuffle(seed=33).train_test_split(test_size=test_size)

In [ ]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["token"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
train_tokenized_datasets = processed_dataset['train'].map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = processed_dataset['test'].map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    from_tf=bool(".ckpt" in model_name),
    cache_dir='./cache_direction',
    )

In [ ]:

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(my_id2label), ignore_mismatched_sizes=True)
model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=17, bias=True)
model.config.id2label = my_id2label
model.config.label2id = my_label2id
model.config._num_labels = len(my_id2label)
model.config.num_labels = len(my_id2label)

In [ ]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=1e-5,
)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: token. If token are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 191
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 120


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.278615,0.924809,0.929197,0.926998,0.925173
2,No log,0.223013,0.933527,0.937956,0.935736,0.934980
3,No log,0.193473,0.950855,0.953285,0.952069,0.952052
4,No log,0.183493,0.955556,0.957299,0.956427,0.956411
5,No log,0.180156,0.968567,0.967153,0.967860,0.967672
6,No log,0.187704,0.969697,0.969343,0.969520,0.968761
7,No log,0.188310,0.970438,0.970438,0.970438,0.969851
8,No log,0.194274,0.970803,0.970803,0.970803,0.970214
9,No log,0.199637,0.970416,0.969708,0.970062,0.969488
10,No log,0.200428,0.970782,0.970073,0.970427,0.969851


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: token. If token are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 34
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: o seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: token. If token are not expected by `BertForTokenClassific

TrainOutput(global_step=120, training_loss=0.11204195817311605, metrics={'train_runtime': 91.1652, 'train_samples_per_second': 20.951, 'train_steps_per_second': 1.316, 'total_flos': 233393637163194.0, 'train_loss': 0.11204195817311605, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: token. If token are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 34
  Batch size = 16


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: o seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 10.0,
 'eval_accuracy': 0.9698510715583001,
 'eval_f1': 0.9704271631982476,
 'eval_loss': 0.20042768120765686,
 'eval_precision': 0.970781592403214,
 'eval_recall': 0.9700729927007299,
 'eval_runtime': 0.6131,
 'eval_samples_per_second': 55.46,
 'eval_steps_per_second': 4.894}

In [ ]:
trainer.save_model('un-ner.model')

Saving model checkpoint to un-ner.model
Configuration saved in un-ner.model/config.json
Model weights saved in un-ner.model/pytorch_model.bin
tokenizer config file saved in un-ner.model/tokenizer_config.json
Special tokens file saved in un-ner.model/special_tokens_map.json


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./un-ner.model/')

Didn't find file ./un-ner.model/added_tokens.json. We won't load it.
loading file ./un-ner.model/vocab.txt
loading file ./un-ner.model/tokenizer.json
loading file None
loading file ./un-ner.model/special_tokens_map.json
loading file ./un-ner.model/tokenizer_config.json


In [ ]:
import torch
paragraph = '''مدیرکل محیطسازمان زیستسازمان استانسازمان البرزسازمان با بیان اینکه با بیان اینکه موضوع شیرابه ##های زبالههای انتقال یافته در منطقهموقعیت حلقهموقعیت درهموقعیت خطری برای این استان است ، گفت : در این مورد گزارشاتی در ۲۵تاریخ مردادتاریخ ۱۳۹۷تاریخ تقدیم مدیران استان شده است .'''
tokens = tokenizer(paragraph)
torch.tensor(tokens['input_ids']).unsqueeze(0).size()

torch.Size([1, 59])

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('./un-ner.model/')
predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
predictions = [label_list[i] for i in predictions]

In [ ]:
words = tokenizer.batch_decode(tokens['input_ids'])
pd.DataFrame({'ner': predictions, 'words': words}).to_csv('un_ner.csv')

In [ ]:
answers = pd.read_csv('./un_ner.csv')
answers